In [ ]:
def format_list(lst: list[str]) -> str:return "'" + "', '".join(lst) + "'"

list1 = ["one", "two", "three"]

formatted_list = format_list(list1)
print(formatted_list)

In [1]:
import os
from openai import OpenAI
from pinecone import Pinecone

from src.embedder import Embedder
from src.db import PineconeDB

# get the keys and create clients
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]   # TODO: where is this actually used???

openai_client = OpenAI(api_key=OPENAI_API_KEY)
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

In [2]:
# index_name = "docu-help"
# namespace = "crawlee"

In [3]:
embedder = Embedder(openai_client)
db = PineconeDB(pinecone_client)

In [4]:
test_index = "testing123"
test_index2 = "testing456"

In [5]:
db.create_index(test_index, embedder)
db.create_index(test_index2, embedder)

Index testing123 already exists
Index testing456 already exists


In [6]:
print(db)

PineconeDB(indexes=[])


In [7]:
db.delete_index(test_index)
db.delete_index(test_index2)

Deleting index testing123...
Deleting index testing456...


In [ ]:
# docs = db.query(index_name, namespace, embedder=embedder, query="how do i code this")

In [ ]:
docs